# MODEL TRAINING

In [1]:
import os 
%pwd

"c:\\Users\\soulo\\MACHINE_LEARNING_UNIVERSE\\PROJECT'S\\Chest-Cancer-Classification-Using-mlflow-and-DVC\\RESEARCH"

In [2]:
os.chdir('../')
%pwd

"c:\\Users\\soulo\\MACHINE_LEARNING_UNIVERSE\\PROJECT'S\\Chest-Cancer-Classification-Using-mlflow-and-DVC"

In [ ]:
# add config.yaml

## Update entity

In [30]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class TrainingConfig:
    root_dir: Path
    trained_model_path: Path
    updated_base_model_path: Path
    training_data: Path
    params_epochs: int
    params_batch_size: int
    params_is_augmentation: bool
    params_image_size: list
    # trained_model: Path
    # params_weight_decay: float


## Update the config manager

In [31]:
from Chest_Cancer_Classification.constants import *
from Chest_Cancer_Classification.utils.common import read_yaml, create_directories
import tensorflow as tf

class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])     
        
    def get_training_config(self) -> TrainingConfig:
        training = self.config.model_trainer
        prepare_foundation_model = self.config.prepare_foundation_model
        params = self.params
        
        # training_data = os.path.join(self.config.data_ingestion.unzip_dir , 'Chest-CT-Scan-data')
        training_data = os.path.join(self.config.data_ingestion.unzip_dir, "Chest-CT-Scan-data")
        
        create_directories([training.root_dir])
        
        training_config = TrainingConfig(
            root_dir=Path(training.root_dir),
            trained_model_path = Path(training.trained_model_path),
            updated_base_model_path = Path(prepare_foundation_model.updated_base_model_path),
            training_data = Path(training_data) ,
            params_epochs= params.EPOCHS,
            params_batch_size=params.BATCH_SIZE,
            params_image_size= params.IMAGE_SIZE,
            params_is_augmentation = params.AUGMENTATION,
        )
        
        return training_config


## Create Components

In [ ]:
import os 
import time
import tensorflow as tf
from zipfile import ZipFile
import urllib.request as request

In [12]:
class Training:
    def __init__(self , config: TrainingConfig):
        self.config = config
        
    def get_base_model(self):
        self.model = tf.keras.models.load_model(self.config.updated_base_model_path)
        
    def train_valid_generator(self):
        '''
        `datagenerator_kwargs` is a dictionary containing the arguments for the ImageDataGenerator. It includes a rescale factor of 1/255 to scale the pixel values to the range [0, 1]. It also sets a validation split of 20%.
        '''
        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split=0.20
        )
        
        
        '''
        dataflow_kwargs is a dictionary containing the arguments for the data flow. It includes a target size for the images, a batch size, and an interpolation method of "bilinear".
        '''
        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )


        ''' 
        The valid_datagenerator is created using the datagenerator_kwargs and the dataflow_kwargs. It reads the images from the training data directory and treats 20% of them as the validation subset.
        '''
        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )



        ''' 
        If image augmentation is enabled (determined by the config.params_is_augmentation parameter), the train_datagenerator is created with the same parameters as the valid_datagenerator, but also includes additional augmentation parameters like rotation, flipping, and zooming. If augmentation is disabled, the train_datagenerator is set to the valid_datagenerator.
        
        The valid_generator and train_generator are created using the respective data generators. They read the images from the training data directory and create batches for the model to train on.
        
        '''
        if self.config.params_is_augmentation:
            train_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
                rotation_range=40,
                horizontal_flip=True,
                width_shift_range=0.2,
                height_shift_range=0.2,
                shear_range=0.2,
                zoom_range=0.2,
                **datagenerator_kwargs
            )
        else:
            train_datagenerator = valid_datagenerator

        self.train_generator = train_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="training",
            shuffle=True,
            **dataflow_kwargs
        )

    
    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        ''' 
        The save_model method is a static method that saves the trained model to a specified path
        '''
        model.save(path)



    
    def train(self):
        ''' 
        The train method performs the actual training of the model. It calculates the number of steps per epoch for both the training and validation data. Then, it uses the model.fit method to train the model on the training data for a specified number of epochs, with the validation data used to evaluate the model's performance during training.
        '''
        self.steps_per_epoch = self.train_generator.samples // self.train_generator.batch_size
        self.validation_steps = self.valid_generator.samples // self.valid_generator.batch_size

        self.model.fit(
            self.train_generator,
            epochs=self.config.params_epochs,
            steps_per_epoch=self.steps_per_epoch,
            validation_steps=self.validation_steps,
            validation_data=self.valid_generator
        )

        self.save_model(
            path=self.config.trained_model_path,
            model=self.model
        )

## Pipeline

In [33]:
try:
    config = ConfigurationManager()
    training_config = config.get_training_config()
    training = Training(config=training_config)
    training.get_base_model()
    training.train_valid_generator()
    training.train()
except Exception as e:
    raise(e)

[2023-12-03 19:26:06,305: INFO: common: yaml file: config\config.yaml loaded successfully]
[2023-12-03 19:26:06,343: INFO: common: yaml file: params.yaml loaded successfully]
[2023-12-03 19:26:06,355: INFO: common: created directory at: artifacts]
[2023-12-03 19:26:06,361: INFO: common: created directory at: artifacts/training]
Found 68 images belonging to 2 classes.
Found 275 images belonging to 2 classes.
Epoch 1/10
17/17 [==============================] - 51s 3s/step - loss: 16.8561 - accuracy: 0.5251 - val_loss: 6.4628 - val_accuracy: 0.3906
Epoch 2/10
17/17 [==============================] - 56s 3s/step - loss: 15.0472 - accuracy: 0.5290 - val_loss: 0.0475 - val_accuracy: 0.9844
Epoch 3/10
17/17 [==============================] - 57s 3s/step - loss: 10.5947 - accuracy: 0.5985 - val_loss: 8.7234 - val_accuracy: 0.3906
Epoch 4/10
17/17 [==============================] - 55s 3s/step - loss: 7.7858 - accuracy: 0.7259 - val_loss: 2.6385 - val_accuracy: 0.8750
Epoch 5/10
17/17 [========